In [32]:
# Para llevar un registro de los cambios
%pip install -q comet_ml
import comet_ml
comet_ml.init(project_name='Tarea 03')

In [33]:
# Librerías
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Input
from tensorflow.keras.optimizers import RMSprop, SGD
from tensorflow.keras import regularizers
from keras.callbacks import ModelCheckpoint

In [34]:
experiment = comet_ml.Experiment(
    auto_histogram_weight_logging=True,
    auto_histogram_gradient_logging=True,
    auto_histogram_activation_logging=True,
    log_code=True,
)
experiment.set_name("Prueba 03.3")

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/gurumasterahb/tarea-03/ada5eead3c8b4d94bae35248d88deb4b
COMET INFO:   Others:
COMET INFO:     Name : Prueba 03.3
COMET INFO:   Parameters:
COMET INFO:     batch_size    : 15
COMET INFO:     epochs        : 40
COMET INFO:     learning_rate : 0.01
COMET INFO:     loss          : categorical_crossentropy
COMET INFO:   Uploads:
COMET INFO:     environment details : 1
COMET INFO:     filename            : 1
COMET INFO:     installed packages  : 1
COMET INFO:     notebook            : 2
COMET INFO:     os packages         : 1
COMET INFO:     source_code         : 1
COMET INFO: 
COMET WARNING: As you are running in a Jupyter environmen

In [35]:
# Carga el dataset de MNIST
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [36]:
# Convierte y normaliza los datos de entrada
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [37]:
# to_categorical convierte las etiquetas (train, test) en matrices one-hot
num_classes = 10
y_trainc = keras.utils.to_categorical(y_train, num_classes)
y_testc = keras.utils.to_categorical(y_test, num_classes)

In [38]:
  # Diccionario con los parámetros que se registrarán en comet
  parameters = {
      'epochs' : 40,
      'batch_size' : 15,
      'learning_rate' : 0.010,
      'loss' : 'categorical_crossentropy',
      'optimizer' : 'adam'
  }
  experiment.log_parameters(parameters)

In [39]:
# Creación del modelo
model = Sequential()
# Es como una "primera capa" de tres capas
model.add(Input(shape = (28,28))) # Matrices de entrada de 28x28 (imáegenes)
model.add(Flatten()) # Convierte las matrices de 28x28 en matrices de 784x1
model.add(Dense(784, activation = 'sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(30, activation = 'relu')) # Segunda capa
model.add(Dense(10, activation = 'softmax')) # Tercera capa

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_3 (Flatten)         (None, 784)               0         
                                                                 
 dense_9 (Dense)             (None, 784)               615440    
                                                                 
 dropout_3 (Dropout)         (None, 784)               0         
                                                                 
 dense_10 (Dense)            (None, 30)                23550     
                                                                 
 dense_11 (Dense)            (None, 10)                310       
                                                                 
Total params: 639300 (2.44 MB)
Trainable params: 639300 (2.44 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [40]:
# Checkpoint que guarda el mejor modelo (cuando la función de costo alcanza un mínimo)
checkpoint = ModelCheckpoint(filepath = 'best_model.hdf5', monitor = 'val_loss', verbose = 1, save_best_only = True, mode = 'min')

In [41]:
# optimizer = RMSprop(learning_rate = 0.0005)
optimizer = 'adam'
model.compile(loss = parameters['loss'], optimizer = optimizer, metrics = ['accuracy'])
model.fit(x_train, y_trainc, batch_size = parameters['batch_size'], epochs = parameters['epochs'], verbose = 1, validation_data = (x_test, y_testc), callbacks = [checkpoint])

COMET INFO: Ignoring automatic log_parameter('verbose') because 'keras:verbose' is in COMET_LOGGING_PARAMETERS_IGNORE


Epoch 1/40
3997/4000 [============================>.] - ETA: 0s - loss: 0.3672 - accuracy: 0.8887
Epoch 1: val_loss improved from inf to 0.16772, saving model to best_model.hdf5
4000/4000 [==============================] - 41s 10ms/step - loss: 0.3673 - accuracy: 0.8887 - val_loss: 0.1677 - val_accuracy: 0.9491
Epoch 2/40
   1/4000 [..............................] - ETA: 51s - loss: 0.0245 - accuracy: 1.0000

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


4000/4000 [==============================] - ETA: 0s - loss: 0.1700 - accuracy: 0.9495
Epoch 2: val_loss improved from 0.16772 to 0.11489, saving model to best_model.hdf5
4000/4000 [==============================] - 34s 8ms/step - loss: 0.1700 - accuracy: 0.9495 - val_loss: 0.1149 - val_accuracy: 0.9642
Epoch 3/40
3999/4000 [============================>.] - ETA: 0s - loss: 0.1247 - accuracy: 0.9630
Epoch 3: val_loss improved from 0.11489 to 0.09287, saving model to best_model.hdf5
4000/4000 [==============================] - 35s 9ms/step - loss: 0.1248 - accuracy: 0.9630 - val_loss: 0.0929 - val_accuracy: 0.9724
Epoch 4/40
3998/4000 [============================>.] - ETA: 0s - loss: 0.0978 - accuracy: 0.9702
Epoch 4: val_loss improved from 0.09287 to 0.08543, saving model to best_model.hdf5
4000/4000 [==============================] - 33s 8ms/step - loss: 0.0978 - accuracy: 0.9702 - val_loss: 0.0854 - val_accuracy: 0.9745
Epoch 5/40
3999/4000 [============================>.] - ETA: 0s

In [42]:
experiment.log_model('MNIST1', 'best_model.hdf5')
experiment.end()

COMET WARNING: Couldn't retrieve Google Colab notebook content
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/gurumasterahb/tarea-03/64e3d945115d42b3a08de5c4b3bd2e30
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [40]                  : (0.888700008392334, 0.994783341884613)
COMET INFO:     batch_accuracy [16000]         : (0.13333334028720856, 1.0)
COMET INFO:     batch_loss [16000]             : (1.445480484107975e-05, 2.7477200031280518)
COMET INFO:     epoch_duration [40]            : (30.788380681000035, 41.587299067000004)
COMET INFO:     loss [40]                      : (0.016448253765702248, 0.3672594428062439)
COMET INFO:     val_accuracy [40]   